# Generating a maritime data_set using a global data_set about the oceans (Shom France)
# It needs a file of size 2gb, it's not on the github

In [1]:
import pandas as pd
import numpy as np

'''
maritime_data = pd.read_csv("./atlantic_data.txt",sep='\t')
maritime_data.columns = ["long", "lat", "depth"]

maritime_data["long"] = maritime_data["long"].astype(float)
maritime_data["lat"] = maritime_data["lat"].astype(float)
maritime_data["depth"] = maritime_data["depth"].astype(float)

maritime_data.drop(maritime_data[maritime_data.lat<44.39767].index, inplace=True)
maritime_data.drop(maritime_data[maritime_data.lat>44.96383].index, inplace=True)
maritime_data.drop(maritime_data[maritime_data.long>-0.72421].index, inplace=True)
maritime_data.drop(maritime_data[maritime_data.long<-1.33169].index, inplace=True)
maritime_data.to_csv(r'maritime_data.csv', index=False, header=True)
'''

'\nmaritime_data = pd.read_csv("./atlantic_data.txt",sep=\'\t\')\nmaritime_data.columns = ["long", "lat", "depth"]\n\nmaritime_data["long"] = maritime_data["long"].astype(float)\nmaritime_data["lat"] = maritime_data["lat"].astype(float)\nmaritime_data["depth"] = maritime_data["depth"].astype(float)\n\nmaritime_data.drop(maritime_data[maritime_data.lat<44.39767].index, inplace=True)\nmaritime_data.drop(maritime_data[maritime_data.lat>44.96383].index, inplace=True)\nmaritime_data.drop(maritime_data[maritime_data.long>-0.72421].index, inplace=True)\nmaritime_data.drop(maritime_data[maritime_data.long<-1.33169].index, inplace=True)\nmaritime_data.to_csv(r\'maritime_data.csv\', index=False, header=True)\n'

# Generating earth data_set using an API (OpenTopoData)

Obtain the coordinates of longitude and latitude

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Number of earth points = num_lin*num_col
var_col=1.329730-0.927928
var_line=44.959459-44.400360

def obtain_lat_and_long(limites_lat=[44.400360,44.959459],limites_long=[-0.927928,-1.329730],num_lin=int(150/var_line),num_col=int(150/var_col)):
    all_coords=[]
    for i in range(num_lin):
        lat=float('%.5f'%(limites_lat[1]-(i)*(limites_lat[1]-limites_lat[0])/num_lin))
        for j in range (num_col):
            long=float('%.5f'%(limites_long[1]-(j)*(limites_long[1]-limites_long[0])/num_col))
            all_coords.append([lat,long])
            
    return all_coords

Utilisation of an API to discover the topography using latitude and longitude

In [42]:
import time
import requests
def get_elevation(coords : list()):
    time.sleep(1.2)
    query =""
    for coord in coords:
        query += '{0},{1}'.format(coord[0],coord[1])
        query += '|'
    query = query[:-1]
    r = requests.get('https://api.opentopodata.org/v1/eudem25m?locations='+query, headers={"Content-Type": "application/json", "Accept": "application/json"}, timeout=1000)
    result = r.json()
    elevations = [res['elevation'] for res in result['results']]
    return elevations

File to save the topography data set

In [43]:
file_data = open("earth_data.csv",'w')
file_data.write("longitude,latitude,altitude,iteration\n")
file_data.close()

Fonction to obtain the topography data of all the points (lat,long)

In [44]:
#Obtain dataset
def dataset_generator(coords,file_name):
    depth=[]
    file_data = open(file_name,'a')
    iter_max=int(len(coords)/100)
    print("Number of iterations", iter_max)
    for i in range(iter_max):
        print("Iter",i+1,"Progress",100*(i+1)/iter_max,"%")
        depth=get_elevation(coords[i*100:(i+1)*100])
        for j in range(len(depth)):
            if depth[j]!=None:
                file_data.write(str(coords[i*100+j][0])+','+str(coords[i*100+j][1])+','+str(depth[j])+','+str(i)+'\n')

Generate the topography data set

In [45]:
coords=obtain_lat_and_long()
dataset_generator(coords,'earth_data.csv')

Number of iterations 971
Iter 1 Progress 0.10298661174047374 %
Iter 2 Progress 0.2059732234809475 %
Iter 3 Progress 0.30895983522142123 %
Iter 4 Progress 0.411946446961895 %
Iter 5 Progress 0.5149330587023687 %
Iter 6 Progress 0.6179196704428425 %
Iter 7 Progress 0.7209062821833162 %
Iter 8 Progress 0.82389289392379 %
Iter 9 Progress 0.9268795056642637 %
Iter 10 Progress 1.0298661174047374 %
Iter 11 Progress 1.132852729145211 %
Iter 12 Progress 1.235839340885685 %
Iter 13 Progress 1.3388259526261586 %
Iter 14 Progress 1.4418125643666324 %
Iter 15 Progress 1.544799176107106 %
Iter 16 Progress 1.64778578784758 %
Iter 17 Progress 1.7507723995880535 %
Iter 18 Progress 1.8537590113285274 %
Iter 19 Progress 1.956745623069001 %
Iter 20 Progress 2.059732234809475 %
Iter 21 Progress 2.1627188465499487 %
Iter 22 Progress 2.265705458290422 %
Iter 23 Progress 2.368692070030896 %
Iter 24 Progress 2.47167868177137 %
Iter 25 Progress 2.5746652935118433 %
Iter 26 Progress 2.677651905252317 %
Iter 27 P

FileNotFoundError: [Errno 2] No such file or directory: './earth_data.xyz'

In [53]:
earth_data = pd.read_csv("./earth_data.csv", sep=",", header=3)
earth_data.columns = ["long", "lat", "depth","Iterations"]
print("Number of points earth_data",len(earth_data['long']))
maritime_data = pd.read_csv("./maritime_data.csv",sep=",")
maritime_data.columns = ["long", "lat", "depth"]

print("Number of points maritime_data :",len(maritime_data["long"]))

Number of points earth_data 68063
Number of points maritime_data : 70256
